# 05 - Model Comparison
Train all three models and compare metrics/plots in one place.


In [1]:
from pathlib import Path
import sys

ROOT = Path("..").resolve()
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

import numpy as np
import pandas as pd


In [2]:
from pathlib import Path

import pandas as pd
import joblib

from src.models import make_model_registry, build_search
from src.eval import evaluate_models, compute_metrics
from src.plots import plot_model_comparison
from _common import load_dataset, prepare_features, ROOT
from src.split import SplitConfig

SEED = 42
TUNE_MODE = "off"  # off | fast | full

# MLflow
MLFLOW_ENABLED = True
MLFLOW_EXPERIMENT = "f1-laptime"
MLFLOW_TRACKING_URI = (ROOT / "mlruns").as_uri()
MLFLOW_RUN_NAME = "model_comparison"

# Verify MLflow availability
if MLFLOW_ENABLED:
    try:
        import mlflow  # noqa: F401
        print(f"MLflow available: {mlflow.__version__}")
    except Exception:
        print("MLflow not installed; set MLFLOW_ENABLED=False or install mlflow.")
        MLFLOW_ENABLED = False

# Model saving / loading
SAVE_MODELS = True
LOAD_MODELS_IF_AVAILABLE = False
REFIT_ON_TRAINVAL = True
MODELS_DIR = ROOT / "reports" / "models"
MODEL_PATHS = {
    "Linear": MODELS_DIR / "linear.joblib",
    "XGBoost": MODELS_DIR / "xgboost.joblib",
    "Deep MLP": MODELS_DIR / "deep_mlp.joblib",
}

split_config = SplitConfig(test_rounds=4)
df, metadata = load_dataset()
train_df, val_df, trainval_df, test_df, features = prepare_features(df, metadata, split_config=split_config)

X_train = train_df[features]
y_train = train_df["LapTimeSeconds"].to_numpy()
X_val = val_df[features]
y_val = val_df["LapTimeSeconds"].to_numpy()
X_trainval = trainval_df[features]
y_trainval = trainval_df["LapTimeSeconds"].to_numpy()

if LOAD_MODELS_IF_AVAILABLE and all(path.exists() for path in MODEL_PATHS.values()):
    fitted = {name: joblib.load(path) for name, path in MODEL_PATHS.items()}
    metrics_rows = []
    preds = {}
    for name, model in fitted.items():
        preds[name] = model.predict(X_val)
        scores = compute_metrics(y_val, preds[name])
        scores["model"] = name
        metrics_rows.append(scores)
    metrics = pd.DataFrame(metrics_rows).sort_values("mae").reset_index(drop=True)
else:
    base_models = make_model_registry(features, random_state=SEED)
    models = {k: build_search(k, v, random_state=SEED, mode=TUNE_MODE) for k, v in base_models.items()}
    metrics, preds, fitted = evaluate_models(models, X_train, y_train, X_val, y_val)

    if SAVE_MODELS:
        MODELS_DIR.mkdir(parents=True, exist_ok=True)
        for name, estimator in fitted.items():
            best = estimator.best_estimator_ if hasattr(estimator, "best_estimator_") else estimator
            if REFIT_ON_TRAINVAL:
                best.fit(X_trainval, y_trainval)
            joblib.dump(best, MODEL_PATHS[name])


Models:   0%|          | 0/3 [00:00<?, ?it/s]

Train size: 40,303 | Eval size: 23,256
Features: 45
Training Linear...


Models:  33%|███▎      | 1/3 [00:00<00:00,  5.34it/s]

Linear -> MAE: 1.6330, R2: 0.9392 (0.2s)
Training XGBoost...


Models:  67%|██████▋   | 2/3 [00:01<00:01,  1.03s/it]

...

c:\Users\tvcar\Desktop\FOM\2. Semester\Maschinelles Lernen\ml_f1\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
                                                     
Models:  67%|██████▋   | 2/3 [00:05<00:01,  1.03s/it]

Using device: cuda (NVIDIA GeForce RTX 2050)


Models: 100%|██████████| 3/3 [01:06<00:00, 22.03s/it]

Early stopping at epoch 78
Deep MLP -> MAE: 1.3197, R2: 0.8930 (64.3s)


,mae,rmse,r2,model
0,0.849126,1.667876,0.972899,XGBoost
1,1.319663,3.313416,0.893045,Deep MLP
2,1.633000,2.498620,0.939179,Linear


In [3]:
plot_model_comparison(metrics)


In [ ]:
# MLflow logging
if MLFLOW_ENABLED:
    try:
        import mlflow
    except ImportError:
        print("MLflow not installed; skipping MLflow logging.")
    else:
        def _coerce_params(params):
            return {k: str(v) for k, v in params.items()}

        if MLFLOW_TRACKING_URI:
            mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
        mlflow.set_experiment(MLFLOW_EXPERIMENT)
        with mlflow.start_run(run_name=MLFLOW_RUN_NAME or "model_comparison"):
            mlflow.log_param("tune_mode", TUNE_MODE)
            mlflow.log_param("test_rounds", split_config.test_rounds)
            mlflow.log_param("loaded_models", LOAD_MODELS_IF_AVAILABLE)

            for model_name, estimator in fitted.items():
                with mlflow.start_run(run_name=model_name, nested=True):
                    if hasattr(estimator, "best_params_"):
                        mlflow.log_params(_coerce_params(estimator.best_params_))
                    elif hasattr(estimator, "get_params"):
                        params = {k: v for k, v in estimator.get_params().items() if k.startswith("model__")}
                        mlflow.log_params(_coerce_params(params))

                    row = metrics[metrics["model"] == model_name].iloc[0]
                    for metric in ("mae", "rmse", "r2"):
                        mlflow.log_metric(f"val_{metric}", float(row[metric]))

                    if SAVE_MODELS and MODEL_PATHS[model_name].exists():
                        mlflow.log_artifact(str(MODEL_PATHS[model_name]), artifact_path="models")
